In [43]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import mltools as ml
import matplotlib.pyplot as plt
%matplotlib inline
import time
from sklearn import *
from sklearn import neural_network
np.random.seed(0)

In [44]:
X = np.genfromtxt("data/X_train.txt",delimiter=None)
Y = np.genfromtxt("data/Y_train.txt",delimiter=None)
Xte = np.genfromtxt('data/X_test.txt',delimiter=None)

In [45]:
# x = X[:10000, :]
# y = Y[:10000]

In [46]:
#comment this for y_submit
# Xtr, Xva, Ytr, Yva = model_selection.train_test_split(
#     x, y, test_size=0.2, random_state=42)

Xtr = X
Ytr = Y

In [47]:
#Build classifiers
logistic_classifier = pipeline.Pipeline([
    ('poly', preprocessing.PolynomialFeatures(degree=2, interaction_only=False)),
    ('logistic', linear_model.LogisticRegression())])
logistic_classifier.fit(Xtr, Ytr)

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('logistic', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [48]:
knn_classifier = neighbors.KNeighborsClassifier(n_neighbors=10, weights="distance",
                                                n_jobs=-1)
knn_classifier.fit(Xtr, Ytr)
      

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=10, p=2,
           weights='distance')

In [49]:
random_forest_classifier = ensemble.RandomForestClassifier(
                    n_estimators=1000, min_samples_leaf= 4, n_jobs=-1,oob_score=True)
random_forest_classifier.fit(Xtr,Ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [50]:
neural_network_classifier = pipeline.Pipeline([
    ("scale", preprocessing.StandardScaler().fit(Xtr)),
    ("nn", neural_network.MLPClassifier(
        hidden_layer_sizes=(14*20, 14*10),
        warm_start=True))])
neural_network_classifier.fit(Xtr, Ytr)
        

Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('nn', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(280, 140), learning_rate='constant',
       learning_r...e=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=True))])

In [51]:
gradient_boosting_classifier = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=0.5,
     max_depth=10, max_leaf_nodes=15, random_state=0)
gradient_boosting_classifier.fit(Xtr,Ytr)
        

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.5, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=15,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [52]:
# max_roc = 0
# index = 0
# for i in range(2,30):
#     classifier_list = [(logistic_classifier, 2),
#                        (knn_classifier, 2),
#                        (random_forest_classifier,27),
#                        (gradient_boosting_classifier, 7),
#                        (neural_network_classifier, 15)
#                       ]
#     y_validation_hat_list = []
#     for classifier in classifier_list:
#         y_validation_hat = classifier[0].predict_proba(Xva)[:,1]
#         for weight in range(classifier[1]):
#             y_validation_hat_list.append(y_validation_hat)
#     y_validation_hat_average = np.mean(np.array(y_validation_hat_list), axis=0)
#     roc = metrics.roc_auc_score(Yva, y_validation_hat_average)
#     if roc > max_roc:
#         max_roc = roc
#         index = i
# print(max_roc,index)

In [53]:
classifier_list = [(logistic_classifier, 2),
                       (knn_classifier, 2),
                       (random_forest_classifier,27),
                       (gradient_boosting_classifier, 7),
                       (neural_network_classifier, 15)
                      ]

In [54]:
# y_validation_hat_list = []
# for classifier in classifier_list:
#     y_validation_hat = classifier[0].predict_proba(Xva)[:,1]
#     for weight in range(classifier[1]):
#         y_validation_hat_list.append(y_validation_hat)
# y_validation_hat_average = np.mean(np.array(y_validation_hat_list), axis=0)
# print("roc:", metrics.roc_auc_score(Yva, y_validation_hat_average))

In [55]:
# submision predict
y_test_hat_list = []
for classifier in classifier_list:
    y_test_hat = classifier[0].predict_proba(Xte)[:,1]
    for weight in range(classifier[1]):
        y_test_hat_list.append(y_test_hat)
y_test_hat_average = np.mean(np.array(y_test_hat_list), axis=0)

In [56]:
np.savetxt("Y_submit.txt",
           np.vstack( (np.arange(len(y_test_hat_average)) , y_test_hat_average) ).T,
           '%d, %.2f',header='ID,Prob1',comments='',delimiter=',');